### Agentic Retrieval in Azure AI Search


### 1. Load Connections

In [10]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os
from IPython.core.display import HTML

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
project_conn_str = os.environ["PROJECT_CONNECTION_STRING"]
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create search agent

In [11]:
import requests

create_agent_request = {
    "name": agent_name,
    "targetIndexes": [ { "indexName": index_name } ],
    "models": [
          {
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "resourceUri": azure_openai_endpoint,
                "apiKey": None,
                "deploymentId": azure_openai_gpt_model,
                "modelName": azure_openai_gpt_model
            }
        }
    ]
}

response = requests.put(
    url=f"{endpoint}/agents/{agent_name}?api-version={api_version}",
    headers={ "Authorization": f"Bearer {token_provider()}" },
    json=create_agent_request
)
response.raise_for_status()


### 3. Create AI Agent

In [12]:
from azure.ai.projects import AIProjectClient
import json
project_client = AIProjectClient.from_connection_string(project_conn_str, credential=credential)

instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name=agent_name,
    instructions=instructions
)
print(json.dumps(agent.as_dict(), indent=2))

{
  "id": "asst_HxHNXsiZm8yrZal58MewhWzk",
  "object": "assistant",
  "created_at": 1745904892,
  "name": "earth-search-agent",
  "description": null,
  "model": "gpt-4o",
  "instructions": "\nAn Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer.\nIf you do not have the answer, respond with \"I don't know\".\n",
  "tools": [],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {},
  "metadata": {},
  "response_format": "auto"
}


### 4. Add Agentic Retrieval tool to AI Agent

In [13]:
from azure.ai.projects.models import FunctionTool, ToolSet, ListSortOrder

thread = project_client.agents.create_thread()

def agentic_retrieval() -> str:
    """
        Searches a NASA e-book about images of Earth at night and other science related facts.
        The returned string is in a JSON format that contains the reference id.
        Be sure to use the same format in your agent's response
    """
    messages = project_client.agents.list_messages(thread.id, order=ListSortOrder.ASCENDING)
    retrieval_request = {
        "messages" : [ { "role": msg.role, "content": [ { "text": msg_content.text.value, "type": msg_content.type } for msg_content in msg.content ] } for msg in messages.data ],
        "targetIndexParams" :  [
            { 
                "indexName" : index_name,
                "rerankerThreshold": 2.5
            } 
        ]
    }
    response = requests.post(
        url=f"{endpoint}/agents/{agent_name}/retrieve?api-version={api_version}",
        headers={ "Authorization": f"Bearer {token_provider()}" },
        json=retrieval_request
    )
    response.raise_for_status()
    result = response.json()
    print("Last Message: {}".format(messages.data[-1].content[0].text.value))
    print("Retrieval Request:")
    print(json.dumps(retrieval_request, indent=2))
    print(json.dumps(result["references"], indent=2))
    print(json.dumps(result["activity"], indent=2))
    return result["response"][0]["content"][0]["text"]

user_functions = { agentic_retrieval }
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

### 5. Start a chat with the agent

In [14]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="""
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value
    display(HTML(output))

Last Message: 
        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    
Retrieval Request:
{
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "\n        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?\n        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?\n    ",
          "type": "text"
        }
      ]
    }
  ],
  "targetIndexParams": [
    {
      "indexName": "earth_at_night",
      "rerankerThreshold": 2.5
    }
  ]
}
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activ

In [15]:
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="How do I find lava at night?"
)

run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, toolset=toolset)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    output = project_client.agents.list_messages(thread_id=thread.id).get_last_text_message_by_role("assistant").text.value
    display(HTML(output))

Last Message: How do I find lava at night?
Retrieval Request:
{
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "text": "\n        Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?\n        Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?\n    ",
          "type": "text"
        }
      ]
    },
    {
      "role": "assistant",
      "content": [
        {
          "text": "The phenomenon of suburban belts displaying larger December brightening than urban cores, despite higher absolute light levels downtown, can be attributed to holiday light displays and activities during Christmas and New Year's. Suburban areas, which tend to have more residential properties, often see significant increases in lighting intensity due to decorative lights used during the